In [10]:
using Flux
using Flux: onehot, onehotbatch, crossentropy, reset!, throttle

In [11]:
corpora = Dict()

for file in readdir("corpus")
  lang = Symbol(match(r"(.*)\.txt", file).captures[1])
  corpus = split(String(read("corpus/$file")), ".")
  corpus = strip.(normalize_string.(corpus, casefold=true, stripmark=true))
  corpus = filter(!isempty, corpus)
  corpora[lang] = corpus
end

corpora

Dict{Any,Any} with 6 entries:
  :en => String["wikipedia (/ˌwɪkɪˈpiːdiə/ ( listen)wik-i-pee-dee-ə or /ˌwɪkiˈp…
  :it => String["wikipedia (pronuncia: vedi sotto) e un'enciclopediaonline a co…
  :so => String["wikipedia (loogu dhawaaqo [ˌwiˑkiˈpidi", "ə])", "waa bar laga …
  :fr => String["wikipedia ecouter est un projet d'encyclopedie universelle, mu…
  :es => String["wikipedia es una enciclopedialibre,[nota 2]\u200bpoliglota y e…
  :da => String["wikipedia er en encyklopædi med abent indhold, skrevet i samar…

In [12]:
langs = collect(keys(corpora))
alphabet = ['a':'z'; '0':'9'; ' '; '\n'; '_'];

In [13]:
# See which chars will be represented as "unknown"
unique(filter(x -> x ∉ alphabet, join(vcat(values(corpora)...))))

169-element Array{Char,1}:
 '('
 '/'
 'ˌ'
 'ɪ'
 'ˈ'
 'ː'
 'ə'
 ' '
 ')'
 '-'
 '['
 ']'
 ','
 ⋮  
 'ব'
 'ল'
 'দ'
 'শ'
 'চ'
 'ট'
 'ম'
 'ঢ'
 'ক'
 'খ'
 'হ'
 'স'

In [15]:
dataset = [(onehotbatch(s, alphabet, '_'), onehot(l, langs))
           for l in langs for s in corpora[l]] |> shuffle
            
train, test = dataset[1:end-100], dataset[end-99:end];

In [19]:
N = 15

scanner = Chain(Dense(length(alphabet), N, σ), LSTM(N, N))
encoder = Dense(N, length(langs))

function model(x)
  state = scanner.(x.data)[end]
  reset!(scanner)
  softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)

loss (generic function with 1 method)

In [20]:
testloss() = mean(loss(t...) for t in test)
opt = ADAM(params(scanner, encoder))
evalcb = () -> @show testloss()

(::#29) (generic function with 1 method)

In [ ]:
Flux.train!(loss, train, opt, cb = throttle(evalcb, 10))

In [22]:
using Interact, Plots

In [32]:
predict(s) =
    isempty(s) ?
        softmax(ones(length(langs))) :
        model(onehotbatch(normalize_string(s, casefold=true, stripmark=true), alphabet, '_')).data

predict (generic function with 1 method)

In [34]:
@manipulate for s = "c'é una bella filosofia"
    bar(String.(langs), predict(s),
        label=["Probability"], ylims=(0,1))
end

Interact.Textbox{AbstractString}(9: "input-4" = c'é una bella filosofia AbstractString , "s", nothing, "c'é una bella filosofia")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 en 
 
 
 it 
 
 
 so 
 
 
 fr 
 
 
 es 
 
 
 da 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Probability